## Cấu trúc DataFrame

**PRIMARY KEY: `bbox_id`**
- Format: `{image_name}_{bbox_index}`
- Ví dụ: `Acbar_u5_0`, `Acbar_u5_1`, `Acbar_u5_2`
- Đảm bảo **duy nhất** cho mỗi bounding box

**Ý nghĩa các cột:**
- `bbox_id`: Khóa chính - ID duy nhất cho mỗi bbox
- `file_path`: Đường dẫn đến file .txt annotation
- `image_name`: Tên file (không có .txt) - dùng để link với ảnh
- `family_name`: Tên họ cá (463 families)
- `bbox_index`: Thứ tự bbox trong file (0=đầu tiên, 1=thứ hai...)
- `x_min, y_min, x_max, y_max`: Tọa độ bounding box

**Ví dụ:**
```
File: Acbar_u5.txt (có 3 dòng = 3 con cá)
→ Tạo 3 rows:
  - bbox_id: Acbar_u5_0 (con cá thứ 1)
  - bbox_id: Acbar_u5_1 (con cá thứ 2)
  - bbox_id: Acbar_u5_2 (con cá thứ 3)
```

In [1]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Đường dẫn đến thư mục Images
images_root = Path(r"D:\HUS\21. nhap_mon_tri_tue_nhan_tao\FishNet-main\Images")

# Kiểm tra thư mục có tồn tại không
if not images_root.exists():
    print(f"Thư mục không tồn tại: {images_root}")
else:
    print(f"Tìm thấy thư mục: {images_root}")
    
    # Đếm số lượng family folders
    family_folders = [f for f in images_root.iterdir() if f.is_dir()]
    print(f"Số lượng Family folders: {len(family_folders)}")

Tìm thấy thư mục: D:\HUS\21. nhap_mon_tri_tue_nhan_tao\FishNet-main\Images
Số lượng Family folders: 463


In [3]:
def get_txt_name(path):
    basename = os.path.basename(path)      # lấy "0a91e3ee-6d71-4106-9841-b25672bda23a.txt"
    name, _ = os.path.splitext(basename)   # tách tên và phần mở rộng
    return name

In [4]:
get_txt_name("Images/Acanthuridae/0a91e3ee-6d71-4106-9841-b25672bda23a.txt")

'0a91e3ee-6d71-4106-9841-b25672bda23a'

In [5]:
def read_annotation_file(txt_path):
    """
    Đọc file annotation FishNet (có thể có nhiều dòng)
    
    Args:
        txt_path: Path đến file .txt
    
    Returns:
        list of dict, mỗi dict là 1 bounding box với các key:
        - bbox_id: ID duy nhất cho mỗi bbox (image_name + bbox_index)
        - image_name: Tên file (không có .txt)
        - family_name: Tên family từ nội dung file
        - bbox_index: Thứ tự bbox trong file (0, 1, 2...)
        - x_min: Tọa độ x góc trên trái
        - y_min: Tọa độ y góc trên trái
        - x_max: Chiều rộng bbox (pixels)
        - y_max: Chiều cao bbox (pixels)
        
        Empty list nếu file rỗng hoặc lỗi
    
    Example:
        >>> read_annotation_file(Path("Acbar_u5.txt"))
        [
            {'bbox_id': 'Acbar_u5_0', 'image_name': 'Acbar_u5', 'bbox_index': 0, 
             'family_name': 'Acheilognathidae', 'x_min': 0, 'y_min': 35, 
             'x_max': 185, 'y_max': 127},
            {'bbox_id': 'Acbar_u5_1', 'image_name': 'Acbar_u5', 'bbox_index': 1,
             'family_name': 'Acheilognathidae', 'x_min': 210, 'y_min': 45, 
             'x_max': 478, 'y_max': 165}
        ]
    """
    try:
        with open(txt_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        
        if not lines:
            return []
        
        image_name = get_txt_name(txt_path)
        results = []
        
        for bbox_index, line in enumerate(lines):
            line = line.strip()
            if not line:  # Bỏ qua dòng trống
                continue
            
            parts = line.split()
            
            # Format: <family_name> <x_min> <y_min> <x_max> <y_max>
            if len(parts) < 5:
                continue
            
            # Tạo bbox_id duy nhất: image_name + "_" + bbox_index
            bbox_id = f"{image_name}_{bbox_index}"
            
            results.append({
                'bbox_id': bbox_id,
                'image_name': image_name,    
                'family_name': parts[0],
                'bbox_index': bbox_index,
                'x_min': int(parts[1]),       
                'y_min': int(parts[2]),        
                'x_max': int(parts[3]),            
                'y_max': int(parts[4])
            })
            
        return results
            
    except:
        return []

In [6]:
rest1 = read_annotation_file("Images/Acanthuridae/0a91e3ee-6d71-4106-9841-b25672bda23a.txt")

In [7]:
rest1

[{'bbox_id': '0a91e3ee-6d71-4106-9841-b25672bda23a_0',
  'image_name': '0a91e3ee-6d71-4106-9841-b25672bda23a',
  'family_name': 'Acanthuridae',
  'bbox_index': 0,
  'x_min': 32,
  'y_min': 29,
  'x_max': 500,
  'y_max': 261}]

In [8]:
rest2 = read_annotation_file("Images/Acheilognathidae/Acbar_u5.txt")
rest2

[{'bbox_id': 'Acbar_u5_0',
  'image_name': 'Acbar_u5',
  'family_name': 'Acheilognathidae',
  'bbox_index': 0,
  'x_min': 0,
  'y_min': 35,
  'x_max': 185,
  'y_max': 127},
 {'bbox_id': 'Acbar_u5_1',
  'image_name': 'Acbar_u5',
  'family_name': 'Acheilognathidae',
  'bbox_index': 1,
  'x_min': 210,
  'y_min': 45,
  'x_max': 478,
  'y_max': 165},
 {'bbox_id': 'Acbar_u5_2',
  'image_name': 'Acbar_u5',
  'family_name': 'Acheilognathidae',
  'bbox_index': 2,
  'x_min': 11,
  'y_min': 150,
  'x_max': 480,
  'y_max': 327}]

In [9]:
def create_fishnet_dataframe(images_root):
    """
    Quét tất cả file .txt trong thư mục Images và tạo DataFrame
    
    Returns:
        DataFrame với các cột:
        - bbox_id: ID duy nhất cho mỗi bbox (PRIMARY KEY)
        - file_path: Đường dẫn đầy đủ
        - image_name: Tên file (không có .txt) - id để phân loại ảnh cá
        - family_name: Tên family - Tên loài
        - bbox_index: Thứ tự bbox trong file (0, 1, 2...)
        - x_min: Tọa độ x góc trên trái
        - y_min: Tọa độ y góc trên trái
        - x_max: Chiều rộng bbox (pixels)
        - y_max: Chiều cao bbox (pixels)
    """
    
    data_list = []
    error_count = 0
    
    # Lấy danh sách tất cả family folders
    family_folders = sorted([f for f in images_root.iterdir() if f.is_dir()])
    
    print(f"Đang quét {len(family_folders)} family folders...")
    
    for family_folder in tqdm(family_folders, desc="Processing"):
        folder_name = family_folder.name
        
        # Lấy tất cả file .txt trong folder
        txt_files = list(family_folder.glob("*.txt"))
        
        for txt_file in txt_files:
            # Đọc annotation (có thể có nhiều bounding boxes)
            annotations = read_annotation_file(txt_file)
            
            if annotations:
                # Mỗi file có thể có nhiều bbox, thêm từng bbox vào data_list
                for annotation in annotations:
                    data_list.append({
                        'bbox_id': annotation['bbox_id'],  # PRIMARY KEY
                        'file_path': str(txt_file),
                        'image_name': annotation['image_name'],
                        'family_name': annotation['family_name'],
                        'bbox_index': annotation['bbox_index'],
                        'x_min': annotation['x_min'],
                        'y_min': annotation['y_min'],
                        'x_max': annotation['x_max'],
                        'y_max': annotation['y_max']
                    })
            else:
                # File rỗng hoặc lỗi format
                error_count += 1
                data_list.append({
                    'bbox_id': None,
                    'file_path': str(txt_file),
                    'image_name': None,
                    'family_name': None,
                    'bbox_index': None,
                    'x_min': None,
                    'y_min': None,
                    'x_max': None,
                    'y_max': None
                })
    
    # Tạo DataFrame
    df = pd.DataFrame(data_list)
    
    # Kiểm tra tính duy nhất của bbox_id
    duplicate_ids = df[df['bbox_id'].notna()]['bbox_id'].duplicated().sum()
    if duplicate_ids > 0:
        print(f"\nCẢNH BÁO: Phát hiện {duplicate_ids} bbox_id trùng lặp!")
    else:
        print(f"\nTất cả bbox_id đều duy nhất (PRIMARY KEY hợp lệ)")

    print(f"Số file lỗi/rỗng: {error_count}")    
    
    return df

In [10]:
# Tạo DataFrame
df_fishnet = create_fishnet_dataframe(images_root)

print("\n" + "="*60)
print("HOÀN THÀNH!")
print("="*60)
print(f"Tổng số bounding boxes: {len(df_fishnet):,}")
print(f"Số lượng unique bbox_id (PRIMARY KEY): {df_fishnet['bbox_id'].nunique():,}")
print(f"Số lượng unique files (.txt): {df_fishnet['file_path'].nunique():,}")
print(f"Số lượng Family folders: {df_fishnet['family_name'].nunique()}")
print(f"Số bbox hợp lệ: {df_fishnet['bbox_id'].notna().sum():,}")
print(f"Số file lỗi/rỗng: {df_fishnet['bbox_id'].isna().sum():,}")

Đang quét 463 family folders...


Processing:   0%|          | 0/463 [00:00<?, ?it/s]

Processing: 100%|██████████| 463/463 [30:37<00:00,  3.97s/it]  




Tất cả bbox_id đều duy nhất (PRIMARY KEY hợp lệ)
Số file lỗi/rỗng: 0

HOÀN THÀNH!
Tổng số bounding boxes: 93,399
Số lượng unique bbox_id (PRIMARY KEY): 93,399
Số lượng unique files (.txt): 84,680
Số lượng Family folders: 463
Số bbox hợp lệ: 93,399
Số file lỗi/rỗng: 0


In [11]:
df_fishnet.head()

,bbox_id,file_path,image_name,family_name,bbox_index,x_min,y_min,x_max,y_max
0,00167b08-4a69-44ed-882d-78b6ba31bcd7_0,D:\HUS\21. nhap_mon_tri_tue_nhan_tao\FishNet-m...,00167b08-4a69-44ed-882d-78b6ba31bcd7,Acanthuridae,0,12,113,74,210
1,00167b08-4a69-44ed-882d-78b6ba31bcd7_1,D:\HUS\21. nhap_mon_tri_tue_nhan_tao\FishNet-m...,00167b08-4a69-44ed-882d-78b6ba31bcd7,Acanthuridae,1,441,230,489,326
2,0057ac91-154f-4b38-990e-ee2d20a9d994_0,D:\HUS\21. nhap_mon_tri_tue_nhan_tao\FishNet-m...,0057ac91-154f-4b38-990e-ee2d20a9d994,Acanthuridae,0,1,28,499,252
3,008a13ed-9cc5-41a9-b420-fa6d84f70ca4_0,D:\HUS\21. nhap_mon_tri_tue_nhan_tao\FishNet-m...,008a13ed-9cc5-41a9-b420-fa6d84f70ca4,Acanthuridae,0,200,133,275,179
4,008a13ed-9cc5-41a9-b420-fa6d84f70ca4_1,D:\HUS\21. nhap_mon_tri_tue_nhan_tao\FishNet-m...,008a13ed-9cc5-41a9-b420-fa6d84f70ca4,Acanthuridae,1,361,130,417,156


In [12]:
df_fishnet.describe()

,bbox_index,x_min,y_min,x_max,y_max
count,93399.000000,93399.000000,93399.000000,93399.000000,93399.000000
mean,0.284864,74.541676,76.863360,474.412895,296.255527
std,1.395998,80.696039,63.637605,136.815642,86.569155
min,0.000000,0.000000,0.000000,3.000000,9.000000
25%,0.000000,17.000000,27.000000,374.000000,238.000000
50%,0.000000,44.000000,61.000000,489.000000,291.000000
75%,0.000000,107.000000,111.000000,609.000000,352.000000
max,31.000000,618.000000,515.000000,787.000000,704.000000


In [13]:
# tổng số file .txt
df_fishnet['file_path'].nunique()

84680

In [14]:
def find_missing_annotations(extract_folder, images_folder):
    """
    So sánh ảnh trong extract/ với annotations trong Images/
    
    Args:
        extract_folder: Đường dẫn đến folder chứa ảnh (.jpg, .png...)
        images_folder: Đường dẫn đến folder chứa annotations (.txt)
    
    Returns:
        DataFrame với:
        - images_without_annotation: Ảnh có trong extract/ nhưng không có .txt
        - annotations_without_image: .txt có trong Images/ nhưng không có ảnh
    """
    # 1. Lấy tất cả ảnh trong extract/ (recursive, tất cả subfolder)
    extract_path = Path(extract_folder)
    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.gif'}
    
    image_files = {}
    for ext in image_extensions:
        for img_path in extract_path.rglob(f'*{ext}'):
            # Key: (folder_name, filename_without_extension)
            folder_name = img_path.parent.name
            file_stem = img_path.stem
            image_files[(folder_name, file_stem)] = str(img_path)
    
    # 2. Lấy tất cả annotations trong Images/
    images_path = Path(images_folder)
    annotation_files = {}
    
    for txt_path in images_path.rglob('*.txt'):
        folder_name = txt_path.parent.name
        file_stem = txt_path.stem
        annotation_files[(folder_name, file_stem)] = str(txt_path)
    
    # 3. So sánh
    image_keys = set(image_files.keys())
    annotation_keys = set(annotation_files.keys())
    
    # Ảnh không có annotation
    missing_annotations = image_keys - annotation_keys
    # Annotation không có ảnh
    missing_images = annotation_keys - image_keys
    
    # 4. Tạo DataFrame kết quả
    result_missing_ann = []
    for folder, stem in missing_annotations:
        result_missing_ann.append({
            'folder': folder,
            'filename': stem,
            'image_path': image_files[(folder, stem)],
            'status': 'No annotation (.txt)'
        })
    
    result_missing_img = []
    for folder, stem in missing_images:
        result_missing_img.append({
            'folder': folder,
            'filename': stem,
            'annotation_path': annotation_files[(folder, stem)],
            'status': 'No image file'
        })
    
    df_missing_ann = pd.DataFrame(result_missing_ann)
    df_missing_img = pd.DataFrame(result_missing_img)
    
    # 5. Summary
    print(f"Tổng kết:")
    print(f"   - Tổng ảnh trong extract/: {len(image_files)}")
    print(f"   - Tổng annotations trong Images/: {len(annotation_files)}")
    print(f"   - Ảnh KHÔNG CÓ annotation: {len(missing_annotations)}")
    print(f"   - Annotation KHÔNG CÓ ảnh: {len(missing_images)}")
    
    return df_missing_ann, df_missing_img


# Sử dụng:
df_no_ann, df_no_img = find_missing_annotations(
    extract_folder="D:/HUS/21. nhap_mon_tri_tue_nhan_tao/fish/extract_zip",
    images_folder='D:/HUS/21. nhap_mon_tri_tue_nhan_tao/FishNet-main/Images'
)

# Xem kết quả:
print("\nẢnh không có annotation:")
print(df_no_ann.head(20))

print("\nAnnotation không có ảnh:")
print(df_no_img.head(20))

# Export để xử lý:
df_no_ann.to_csv('images_missing_annotations.csv', index=False)
df_no_img.to_csv('annotations_missing_images.csv', index=False)

Tổng kết:
   - Tổng ảnh trong extract/: 94805
   - Tổng annotations trong Images/: 84680
   - Ảnh KHÔNG CÓ annotation: 10125
   - Annotation KHÔNG CÓ ảnh: 0

Ảnh không có annotation:
            folder                              filename  \
0       Cyprinidae  a80b3f4d-a3ac-4af7-b519-a866249703f2   
1       Cyprinidae  ac3a3847-09ca-4dd1-9339-c6989c54695b   
2      Enoplosidae  175eb728-5fa0-4cca-a665-f11b5255be68   
3       Dasyatidae  27eecc98-6e29-4e50-b652-07140c4ba02a   
4       Cyprinidae  4208f226-b0c6-41b8-a3f0-7f62849e44b8   
5    Centrarchidae  532d3f4d-0ad7-45cc-9ace-bfe11f08b907   
6       Dasyatidae  a3d2c971-449d-407e-a805-828f1daeb436   
7         Labridae  5e68f588-f1f7-4151-8ee5-2dbe26b0d8fb   
8       Girellidae  e83163f6-6ae5-45e7-a388-d666834f0aa7   
9          Molidae  326510c9-f2b1-40c3-a758-4df83076f2bb   
10     Pempheridae                              Pekur_u0   
11      Salmonidae  d9ac40df-9fff-423e-b197-09e84bc58dd2   
12   Pomacentridae  06da8aa2-4ab1-46a

In [15]:
# Tính tọa độ bbox_width, bbox_height
df_fishnet['bbox_width'] = df_fishnet['x_max'] - df_fishnet['x_min']
df_fishnet['bbox_height'] = df_fishnet['y_max'] - df_fishnet['y_min']

In [16]:
# export df_fishnet to CSV
df_fishnet.to_csv('fishnet_annotations.csv', index=False)

In [17]:
# export các ảnh không có annotation
df_no_ann.to_csv('images_missing_annotations.csv', index=False)

In [18]:
print(f"Tổng số rows: {len(df_fishnet)}")
print(f"Số bbox_id duy nhất: {df_fishnet['bbox_id'].nunique()}")
print(f"Có trùng lặp bbox_id: {df_fishnet['bbox_id'].duplicated().sum() > 0}")

Tổng số rows: 93399
Số bbox_id duy nhất: 93399
Có trùng lặp bbox_id: False


In [19]:
# Tìm các file có nhiều bounding boxes
multi_bbox_files = df_fishnet.groupby('file_path').size()
multi_bbox_files = multi_bbox_files[multi_bbox_files > 1].sort_values(ascending=False)
print(f"Số files có nhiều bbox: {len(multi_bbox_files)}")
print(f"\nTop 10 files có nhiều bbox nhất:")
print(multi_bbox_files.head(10))

Số files có nhiều bbox: 4136

Top 10 files có nhiều bbox nhất:
file_path
D:\HUS\21. nhap_mon_tri_tue_nhan_tao\FishNet-main\Images\Acanthuridae\2aa0d89e-08d0-4dea-a5d2-f97285278139.txt    32
D:\HUS\21. nhap_mon_tri_tue_nhan_tao\FishNet-main\Images\Poeciliidae\dec7557b-83d4-4999-89e4-50ffcf1aadcc.txt     31
D:\HUS\21. nhap_mon_tri_tue_nhan_tao\FishNet-main\Images\Lutjanidae\8b6d0580-1123-4d30-9ee8-6407496e413f.txt      27
D:\HUS\21. nhap_mon_tri_tue_nhan_tao\FishNet-main\Images\Mugilidae\Livai_u3.txt                                   27
D:\HUS\21. nhap_mon_tri_tue_nhan_tao\FishNet-main\Images\Aulostomidae\1c9aabca-f453-4ae8-a276-d070648baf25.txt    25
D:\HUS\21. nhap_mon_tri_tue_nhan_tao\FishNet-main\Images\Acanthuridae\b9b02291-5ef3-4216-bb29-b6865eda6c00.txt    25
D:\HUS\21. nhap_mon_tri_tue_nhan_tao\FishNet-main\Images\Salmonidae\c8247b90-5d61-4bc7-b0c9-bc424e9acfc4.txt      25
D:\HUS\21. nhap_mon_tri_tue_nhan_tao\FishNet-main\Images\Mullidae\1e6d6552-2354-4a2c-9dc0-b5e845936430.txt  

## Xóa ảnh không có annotation

**Mục đích:** Chỉ giữ lại những ảnh có file .txt annotation tương ứng

**Logic:**
1. Quét tất cả file `.txt` trong `Images/`
2. Với mỗi `.txt`, tìm ảnh tương ứng trong `extract_zip/`
3. Copy ảnh sang folder mới `filtered_images/`
4. Kết quả: Folder mới chỉ chứa ảnh có annotation

**Lưu ý:**
- Không xóa trực tiếp folder gốc (an toàn)
- Giữ nguyên cấu trúc thư mục (family folders)
- Hỗ trợ nhiều định dạng ảnh (.jpg, .png, .jpeg...)

In [22]:
import shutil

def copy_images_with_annotations(extract_folder, images_folder, output_folder):
    """
    Copy ảnh từ extract_folder sang output_folder, CHỈ GIỮ những ảnh có annotation .txt
    
    Args:
        extract_folder: Folder chứa ảnh gốc (extract_zip)
        images_folder: Folder chứa file .txt annotations
        output_folder: Folder mới để lưu ảnh đã lọc
    
    Process:
        1. Quét tất cả file .txt trong Images/
        2. Với mỗi .txt, tìm ảnh tương ứng trong extract_folder
        3. Copy ảnh sang output_folder (giữ nguyên cấu trúc thư mục)
    """
    extract_path = Path(extract_folder)
    images_path = Path(images_folder)
    output_path = Path(output_folder)
    
    # Đếm số lượng
    total_annotations = 0
    images_found = 0
    images_not_found = 0
    
    print("Bước 1: Quét tất cả file .txt trong Images/...")
    
    # Lấy tất cả file .txt
    txt_files = list(images_path.rglob('*.txt'))
    total_annotations = len(txt_files)
    
    print(f"Tìm thấy {total_annotations} file .txt annotations")
    print(f"\nBước 2: Tìm và copy ảnh tương ứng...")
    
    # Với mỗi file .txt, tìm ảnh tương ứng
    for txt_path in tqdm(txt_files, desc="Copying images"):
        # Lấy tên folder (family) và tên file (không có .txt)
        family_name = txt_path.parent.name  # Ví dụ: "Acanthuridae"
        image_name = txt_path.stem          # Ví dụ: "0a91e3ee-6d71-4106-9841-b25672bda23a"
        
        # Tìm ảnh trong extract_folder
        # Thử các extension: .jpg, .jpeg, .png, .bmp, .gif
        image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']
        
        source_image_path = None
        for ext in image_extensions:
            # Đường dẫn ảnh: extract_folder/family_name/image_name.ext
            potential_path = extract_path / family_name / f"{image_name}{ext}"
            if potential_path.exists():
                source_image_path = potential_path
                break
        
        if source_image_path:
            # Tạo folder đích (nếu chưa tồn tại)
            dest_folder = output_path / family_name
            dest_folder.mkdir(parents=True, exist_ok=True)
            
            # Copy ảnh
            dest_image_path = dest_folder / source_image_path.name
            shutil.copy2(source_image_path, dest_image_path)
            
            images_found += 1
        else:
            images_not_found += 1
            # In ra để debug (có thể comment nếu quá nhiều)
            # print(f"Không tìm thấy ảnh: {family_name}/{image_name}")
    
    # Báo cáo kết quả
    print(f"\n{'='*60}")
    print("HOÀN THÀNH!")
    print(f"{'='*60}")
    print(f"Tổng số annotations (.txt): {total_annotations}")
    print(f"Ảnh tìm thấy và đã copy: {images_found}")
    print(f"Ảnh KHÔNG tìm thấy: {images_not_found}")
    print(f"\nFolder mới: {output_path}")
    print(f"{'='*60}")
    
    return {
        'total_annotations': total_annotations,
        'images_found': images_found,
        'images_not_found': images_not_found
    }


# SỬ DỤNG:
result = copy_images_with_annotations(
    extract_folder="D:/HUS/21. nhap_mon_tri_tue_nhan_tao/FishNet-main/extract_zip/Image_Library",
    images_folder="D:/HUS/21. nhap_mon_tri_tue_nhan_tao/FishNet-main/Images",
    output_folder="D:/HUS/21. nhap_mon_tri_tue_nhan_tao/FishNet-main/filtered_images"
)

Bước 1: Quét tất cả file .txt trong Images/...
Tìm thấy 84680 file .txt annotations

Bước 2: Tìm và copy ảnh tương ứng...
Tìm thấy 84680 file .txt annotations

Bước 2: Tìm và copy ảnh tương ứng...


Copying images: 100%|██████████| 84680/84680 [24:24<00:00, 57.80it/s]  




HOÀN THÀNH!
Tổng số annotations (.txt): 84680
Ảnh tìm thấy và đã copy: 84680
Ảnh KHÔNG tìm thấy: 0

Folder mới: D:\HUS\21. nhap_mon_tri_tue_nhan_tao\FishNet-main\filtered_images
